<a href="https://colab.research.google.com/github/hndrr/colab/blob/main/spotipy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

参考: 
*   https://www.hamlet-engineer.com/posts/spotify01.html
*   https://qiita.com/ysk0832/items/633a2ebedf26d78951a8

In [ ]:
#　ライブラリのInstall
# https://spotipy.readthedocs.io/
!pip install spotipy
!pip install python-dotenv

In [ ]:
import json
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth


In [ ]:
# JSONを整形して出力する関数を定義
def pretty_print_json(json_data):
    print(json.dumps(json_data, indent=2, ensure_ascii=False))

In [ ]:
import os
from dotenv import load_dotenv

# ファイルのパスを指定
# .envに環境変数を書き込む必要あり
file_path = '/content/drive/MyDrive/Colab Notebooks/.env'

# .envファイルを読み込む
load_dotenv(file_path)

# 環境変数の取得
CLIENT_ID = os.environ.get('CLIENT_ID')
CLIENT_SECRET = os.environ.get('CLIENT_SECRET')
REDIRECT_URL = os.environ.get('REDIRECT_URL')
USERNAME = os.environ.get('USERNAME')

# 環境変数の表示
print(f"CLIENT_ID: {CLIENT_ID}")
print(f"CLIENT_SECRET: {CLIENT_SECRET}")
print(f"REDIRECT_URL: {REDIRECT_URL}")
print(f"USERNAME: {USERNAME}")


In [ ]:
scope = "user-library-read"
sp_oauth = SpotifyOAuth(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    redirect_uri=REDIRECT_URL,
    scope=scope
)

auth_url = sp_oauth.get_authorize_url()

print(auth_url)

auth_url にアクセスしてcodeの値をコピーして↓入力する

In [ ]:
code = ""#@param{type: "string"}
token_info = sp_oauth.get_access_token(code=code)
REFRESH_TOKEN = token_info['refresh_token']
ACCESS_TOKEN = token_info['access_token']
print(REFRESH_TOKEN)
print(ACCESS_TOKEN)

In [ ]:
# Podcast IDを指定
podcast_id = '6h7WBgX1XlTuypPKhyL3qI'#@param{type: "string"}

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth

scope = "user-library-read"
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=CLIENT_ID,
        client_secret=CLIENT_SECRET, 
        redirect_uri=REDIRECT_URL, scope=scope))

# Podcast情報を取得
podcast_data = sp.show(podcast_id)

# Podcastのエピソード情報を取得
episodes_data = sp.show_episodes(podcast_id)

# 結果を表示
print('Show title:', podcast_data['name'])
print('Description:', podcast_data['description'])
print('Total episodes:', podcast_data['total_episodes'])

print('Episodes:')
for episode in episodes_data['items']:
    print(episode['name'], '-', episode['release_date'], '-', episode['duration_ms'])


In [ ]:
# エンドポイントを叩いてJSONを拾えるか確認
# https://developer.spotify.com/documentation/web-api/reference/get-a-shows-episodes
import requests
import json

# Spotify APIエンドポイント
SHOW_ENDPOINT = 'https://api.spotify.com/v1/shows/{id}'
EPISODES_ENDPOINT = 'https://api.spotify.com/v1/shows/{id}/episodes'

HEADERS = {
    'Authorization': 'Bearer ' + ACCESS_TOKEN
}

# Podcast情報を取得
show_response = requests.get(SHOW_ENDPOINT.format(id=podcast_id), headers=HEADERS)
show_data = json.loads(show_response.text)

# Podcastのエピソード情報を取得
episodes_response = requests.get(EPISODES_ENDPOINT.format(id=podcast_id), headers=HEADERS)
episodes_data = json.loads(episodes_response.text)

pretty_print_json(episodes_data)

In [ ]:
# APIのIDとキー
# https://developer.spotify.com/dashboard
def auth_spotify():
    client_credentials_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
    return spotipy.Spotify(client_credentials_manager=client_credentials_manager, language="en")

def auth_spotify2():
    auth_manager = SpotifyOAuth(client_id=CLIENT_ID,
        client_secret=CLIENT_SECRET, 
        redirect_uri=REDIRECT_URL, 
        scope='user-follow-modify',
        open_browser=False)
    return spotipy.Spotify(auth_manager=auth_manager, language="en")

In [ ]:
# spotifyの検索キーを https://open.spotify.com/ のURLからコピペしてくる
artist_id = '7kOrrFIBIBc8uCu2zbxbLv'#@param{type: "string"}
serach_key = 'spotify:artist:'#@param{type: "string"}
spotify = auth_spotify()
results = spotify.artist_top_tracks(artist_id)
#　取得件数
num = 20#@param{type: "number"}

# 取得
for track in results['tracks'][:num]:
    print('track    : ' + track['name'])
    print('audio    : ' + track['preview_url'])
    print('cover art: ' + track['album']['images'][0]['url'])

In [ ]:
# 認証
import spotipy
import spotipy.util as util
token = util.prompt_for_user_token(
    scope='user-follow-modify',
        username=USERNAME,
        redirect_uri=REDIRECT_URL,
        client_id=CLIENT_ID,
        client_secret=CLIENT_SECRET) 
sp = spotipy.Spotify(auth=token)
print('ok')

In [ ]:
sp = auth_spotify2()

token = util.prompt_for_user_token(
    scope='user-follow-modify',
        username=USERNAME,
        redirect_uri=REDIRECT_URL,
        client_id=CLIENT_ID,
        client_secret=CLIENT_SECRET) 

if token:
    sp = spotipy.Spotify(auth=token)
    podcast = sp.podcast(podcast_id)
    print(podcast)
else:
    print("Can't get token for", USERNAME)


  # response = sp.current_user_saved_tracks()
  # print(response)

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import time

def create_spotify():
    auth_manager = SpotifyOAuth(
        scope='user-follow-modify',
        username=USERNAME,
        redirect_uri=REDIRECT_URL,
        client_id=CLIENT_ID,
        client_secret=CLIENT_SECRET)

    spotify = spotipy.Spotify(auth_manager=auth_manager)

    return auth_manager, spotify

def refresh_spotify(auth_manager, spotify):
    token_info = auth_manager.cache_handler.get_cached_token()
    if auth_manager.is_token_expired(token_info):
        auth_manager, spotify = create_spotify()
    return auth_manager, spotify

if __name__ == '__main__':
    auth_manager, spotify = create_spotify()

    while True:
        auth_manager, spotify = refresh_spotify(auth_manager, spotify)
        playing = spotify.currently_playing()
        if playing:
            print(playing['item']['name'])
        else:
            print('Nothing is playing.')
        time.sleep(30)


In [ ]:
import spotipy  
from spotipy.oauth2 import SpotifyOAuth  
  
sp_oauth = SpotifyOAuth(client_id=CLIENT_ID, client_secret=CLIENT_SECRET, redirect_uri=REDIRECT_URL, scope='user-follow-modify',)  
  
# access_token = sp_oauth.get_cached_token()  
refresh_token = sp_oauth.get_refresh_token()  


In [ ]:
%env